# Packages


In [9]:
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tqdm import tqdm
import json
import duden
import time
from collections import Counter
import pandas as pd
import csv


# Get Lyrics


- API Schlüssel und Zugangsdaten einsetzen
- Playlist IDs von Spotify suchen 
- Jeder Track der Playlist wird auf LyricsGenius gesucht
- Sollte zu einem Song kein Lyrics gefunden werden, wird die Anzahl hochgezählt
- Es wird eine JSON Datei gebildet

In [6]:
SPOTIPY_CLIENT_ID = '46da6046625f409cb53ab06bf807861d'
SPOTIPY_CLIENT_SECRET = '00efb0ab68db4f1eaf27ec53a4c1e068'
GENIUS_ACCESS_TOKEN = 'OsBvEsJ_OLKYR42CJoaA_N97GNOS44XXFk4KPPcLtCYnWmDwnnzY2RpcG_SdG25a'

Ausführen von getlyrics.py

## Analysis

In [10]:
with open("./all_songs_neu2.json") as f:
    songs_dict = json.load(f)

encoded= json.dumps (songs_dict, ensure_ascii=True)
songs_dict_0 =json.loads (encoded)

1. Spacy

Ausführen von cyspa.py

Hier werden alle Nomen ausgegeben in einer csv datei: SongID, Nomen, Geschlecht, Release Year, Artist

In [11]:
#NEU #Spacy muss gedownloaded werden mit : pip install -U pip setuptools wheel   und    pip install spacy 
#de_core_news_sm-3.5.0-py3-none-any.whl (muss gedownloaded werden -> https://spacy.io/models/de/)
#de_core_news_lg-3.5.0-py3-none-any.whl
#de_dep_news_trf-3.5.0-py3-none-any.whl
import csv 



nlp2 = spacy.load ("de_core_news_lg")
playlist="Spotify"
song_id=0

with open ('Nomen_csv.csv','w', newline ='') as nomen_file:
    writer = csv.writer(nomen_file, delimiter=';', lineterminator='\n')
    writer.writerow (["ID","Noun", "Gender", "Year", "Artist"])    

    for song_id in tqdm(range(len(songs_dict_0[playlist]))):

        lyrics_from_song = songs_dict_0[playlist][str(song_id)]["lyrics"] #hier wird aus dem der Datei mit allen Songtexten nur das Lyrics abgefragt
        year_from_song = songs_dict_0[playlist][str(song_id)]["release_year"]
        artist_song=songs_dict_0[playlist][str(song_id)]["artist_name"]
        doc3 = nlp2 (lyrics_from_song)

        for token in doc3: 
            if token.pos_ == "NOUN":
                    try:
                        row= ([song_id, token.text,token.morph.get('Gender'), year_from_song, artist_song])
                        writer.writerow(row)
                    except:
                        print(f"Error at Song {song_id} with word {token.text}.")
                        pass
            else:
                pass


 11%|█▏        | 10/88 [00:00<00:04, 16.93it/s]

Error at Song 6 with word jеdem.
Error at Song 6 with word wеrden.


 42%|████▏     | 37/88 [00:01<00:02, 21.41it/s]

Error at Song 34 with word siе.


 68%|██████▊   | 60/88 [00:02<00:01, 24.39it/s]

...

 82%|████████▏ | 72/88 [00:03<00:00, 20.83it/s]

Error at Song 67 with word Совета.
Error at Song 67 with word против.


 92%|█████████▏| 81/88 [00:03<00:00, 21.70it/s]

Error at Song 78 with word hеute.


100%|██████████| 88/88 [00:04<00:00, 20.78it/s]


TODO :
- Segementierung mit SPACY

In [ ]:
##csv datei sortieren (funktioniert noch nicht)

import pandas as pd

df=pd.read_csv('Nomen_csv.csv',encoding ='ISO-8859-1')
sorted_df=df.sort_values (by=['2'])
sorted_df.to_csv('sorted_filename.csv', index=False)

NOMEN

In [24]:
scrape_data = False # decide, whether to scrape full data again or read an exisiting json file.

### Mit Genderliste vergleichen

TODO:
- mit Genderliste Vergleichen
- mit Ausdrucks-Worliste vergleichen
- mit Frau/Mann List vergleichen

In [ ]:
### Im folgenden erfolgt die Abfrage mit der genderneutralen Wortliste und der Nomenliste
import pandas as pd

gender_neu = pd.read_csv("./gender_neu_Nomen.csv",sep=";",header=None,encoding="utf8")
gender_neu = gender_neu.iloc[:, 0]
gender_neu.columns = ["Wort"]
gender_neu = list(gender_neu)


In [ ]:
pd.read_csv('Nomen_csv.csv', sep=';', encoding ='ISO-8859-1', usecols= [3])

In [10]:
complete_df = pd.DataFrame(columns=["Wort","neutral", "Jahr"])
tmp = []
playlist='Prototyp'
for id in nomen_sortiert[playlist].keys():
    for g in nomen_sortiert[playlist][id].keys(): # geschlecht
        w = nomen_sortiert[playlist][id][g] # wort
        is_gender_neutral = True if w in gender_neu else False

        complete_df.loc[len(complete_df)] = [g, w, is_gender_neutral]
        tmp.append(g)

In [11]:
freq = Counter(tmp)
freq

Counter({'Maskuline Nomen': 1195,
         'Feminine Nomen': 1092,
         'Neutrale Nomen': 711})

In [14]:
complete_df.head()

,Geschlecht,Wort,neutral
0,Feminine Nomen,Kneipe,False
1,Feminine Nomen,Ecke,False
2,Maskuline Nomen,Abend,False
3,Feminine Nomen,Liebe,False
4,Feminine Nomen,Freundschaft,False


In [15]:
complete_df.to_json("./test.json")

In [ ]:
complete_df.loc[complete_df["neutral"] == True, :]
complete_df

In [94]:
with open('Example.csv', 'w', newline = '') as csvfile:
    my_writer = csv.writer(csvfile, delimiter = ' ')
    my_writer.writerow(complete_df)

In [7]:
import spacy
from spacy.lang.de.examples import sentences 

nlp = spacy.load("de_core_news_lg")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, )

Die ganze Stadt ist ein Startup: Shenzhen ist das Silicon Valley für Hardware-Firmen
Die DET ['Fem']
ganze ADJ ['Fem']
Stadt NOUN ['Fem']
ist AUX []
ein DET ['Neut']
Startup NOUN ['Neut']
: PUNCT []
Shenzhen PROPN ['Neut']
ist AUX []
das DET ['Neut']
Silicon PROPN ['Neut']
Valley PROPN ['Neut']
für ADP []
Hardware-Firmen NOUN ['Fem']
